# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.




In [191]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators

In [24]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

display_results (generic function with 1 method)

In [25]:
function Compute_AlphaAff(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 1 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end

function Compute_AlphaMax(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 100/99 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end


Compute_AlphaMax (generic function with 1 method)

In [556]:

function MehrotraPCQuad(QM, max_iter, eps=1e-10, tol_step_x=1e-8, eps_mu=1e-8, alpha_step=1e-2, display=true)
    
    # get variables from QuadraticModel
    x_0 = QM.meta.x0
    @assert all(x_0 .> 0)
    A = Matrix(sparse(QM.data.Arows, QM.data.Acols, QM.data.Avals))
    n_rows, n_cols = size(A) 
    Q = Matrix(sparse(QM.data.Hrows, QM.data.Hcols, QM.data.Hvals))
    c = QM.data.c
    c0 = QM.data.c0
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    s_0 = ones(n_cols)
    lambda_0 = A'\(c-s_0) # least square initialisation
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    
    rb_0 = A * x_0 - b
    rc_0 = -Q * x_0 + A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk = Diagonal(x_k)
    Sk = Diagonal(s_k)
    mu_k = copy(mu_0)
    rb_k = copy(rb_0)
    rc_k = copy(rc_0)
    k = 0
    pdd, cond_rb, cond_rc = 1., 1., 1.
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    
    
    while k<=max_iter && opti_pdd==false && opti_rb==false && opti_rc==false && small_step_x==false && small_mu==false
        
        # Affine scaling direction
        Jacob_Fk = [-Q A' I
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk_aff = [-rc_k
                  -rb_k
                  -x_k.*s_k]
        dir_aff_k = Jacob_Fk\Fk_aff
        

        alpha_aff_pri = Compute_AlphaAff(alpha_step, x_k, dir_aff_k[1:n_cols])
        alpha_aff_dual = Compute_AlphaAff(alpha_step, s_k, dir_aff_k[n_rows+n_cols+1: end])
        mu_aff = (x_k + alpha_aff_pri * dir_aff_k[1:n_cols])' * 
                    (s_k + alpha_aff_dual * dir_aff_k[n_rows+n_cols+1: end]) / n_cols
        sigma = (mu_aff / mu_k)^3
        
        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e - dir_aff_k[1:n_cols].*dir_aff_k[n_rows+n_cols+1: end]] 
        dir_cc_k = Jacob_Fk\Fk_cc
         
        dir_k = dir_aff_k .+ dir_cc_k # final direction
         
        alpha_max_pri = Compute_AlphaMax(alpha_step, x_k, dir_k[1:n_cols])
        alpha_max_dual = Compute_AlphaMax(alpha_step, s_k, dir_k[n_rows+n_cols+1: end])
        
        # new parameters
        alpha_k_pri, alpha_k_dual = min(0.99*alpha_max_pri, 1), min(0.99*alpha_max_dual, 1)
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_k_dual * dir_k[n_cols+1: n_rows+n_cols]
        s_k += alpha_k_dual * dir_k[n_rows+n_cols+1: end]
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        
        k += 1
        
        if display == true
            @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
    end
    
    if display == true
        criteria = [k > max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuad (generic function with 8 methods)

In [557]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3];
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1; 2; 3];

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [558]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [559]:
res_mpc1 =  MehrotraPCQuad(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.75e+01    |         3.30e+00       |    1.12e+00  |    2.20e+00  |  0.00e+00  |  2.00e+00
   1 |      1.23e+01    |         3.22e-02       |    6.65e-02  |    7.64e-02  |  3.08e+00  |  2.51e-01
   2 |      1.35e+01    |         2.25e-03       |    1.98e-03  |    2.29e-03  |  2.40e-01  |  6.43e-03
   3 |      1.35e+01    |         2.45e-05       |    3.94e-05  |    4.66e-05  |  4.94e-03  |  9.70e-05
   4 |      1.35e+01    |         2.76e-07       |    7.85e-07  |    9.86e-07  |  9.82e-05  |  1.62e-06
   5 |      1.35e+01    |         3.38e-09       |    1.56e-08  |    2.04e-08  |  1.95e-06  |  2.92e-08
   6 |      1.35e+01    |         4.61e-11       |    3.11e-10  |    4.15e-10  |  3.89e-08  |  5.50e-10

 stopping criterion = ["pdd <= eps", "mu_k <= eps_mu"]

---------

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [5.1454e-10, 3.0, 5.97718e-10]
  "lambda_opt" => [-9.93575, 12.0]
  "s_opt"      => [1.93575, 3.12465e-11, 0.935748]
  "n_iter"     => 6
  "pdd"        => 4.61333e-11
  "cond_rb"    => 3.10886e-10
  "cond_rc"    => 4.14559e-10

In [560]:
@benchmark MehrotraPCQuad(SM, 20, 1e-10, 1e-8, 1e-8, 1e-2, false)

BenchmarkTools.Trial: 
  memory estimate:  944.25 KiB
  allocs estimate:  10650
  --------------
  minimum time:     632.101 μs (0.00% GC)
  median time:      646.800 μs (0.00% GC)
  mean time:        761.476 μs (13.72% GC)
  maximum time:     6.606 ms (87.67% GC)
  --------------
  samples:          6534
  evals/sample:     1